<a href="https://colab.research.google.com/github/fahimabrar/Natural-Language-Processing/blob/main/Multiclass%20Classification%20of%20Text/Toxic_Comment_Classification_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import LSTM, GRU, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score
import pickle

In [8]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
# maximum size for an input/comment/sentence/line 
MAX_VOCAB_SIZE = 20000
# max vocab size is 20000 selected, If there are more than 20000 unique words in the dataset it will be truncated automatically by keras
EMBEDDING_DIM = 200
# we will use gloVe pretrained embeddings, the dimension is 200
VALIDATION_SPLIT = 0.2
# 20 percent data is selected for validating the model
num_words = 20000

In [9]:
data_path = '/content/drive/My Drive/NLP/Toxic_comment/'

# path is defined for loading the data

In [10]:
with open(data_path +"/embedding_matrix.pkl", "rb") as encoded_pickle:
  embedding_matrix = pickle.load(encoded_pickle)

with open(data_path +"/targets.pkl", "rb") as encoded_pickle:
  targets = pickle.load(encoded_pickle)

with open(data_path +"/data.pkl", "rb") as encoded_pickle:
  data = pickle.load(encoded_pickle)

In [11]:
targets.shape

(159571, 6)

In [12]:
embedding_matrix.shape

(20000, 200)

In [13]:
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=False
)

# we made trainalbe false as we are using pretrained weights from GloVe embeddings

In [18]:
# defining the model woth keras functional API
input = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input)
x = LSTM(128, return_sequences = True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(6, activation='sigmoid')(x)

model = Model(input, output)

In [19]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 200)          4000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          168448    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 774       
Total params: 4,185,734
Trainable params: 185,734
Non-trainable params: 4,000,000
___________________________________________

In [20]:
model.compile(
  loss='binary_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [21]:
BATCH_SIZE = 128
# arbritary selected
EPOCHS = 3
# arbritary selected

r = model.fit(
  data,
  targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

Epoch 1/3
998/998 [==============================] - 339s 338ms/step - loss: 0.0631 - accuracy: 0.9825 - val_loss: 0.0551 - val_accuracy: 0.9852
Epoch 2/3
998/998 [==============================] - 339s 340ms/step - loss: 0.0477 - accuracy: 0.9828 - val_loss: 0.0486 - val_accuracy: 0.8713
Epoch 3/3
998/998 [==============================] - 338s 339ms/step - loss: 0.0439 - accuracy: 0.9813 - val_loss: 0.0463 - val_accuracy: 0.9936
